# Project description

We will be working with the BoolQ corpus. It consists of yes/no questions, passages from Wikipedia containing answers to the question, titles of the source articles and the answers. 

The quesitons are gathered from google searches, where Wikepidia page was one of the first five search results.

Corpus is described in the paper:
Christopher Clark, Kenton Lee, Ming-Wei Chang, Tom Kwiatkowski, Michael Collins, Kristina Toutanova BoolQ: Exploring the Surprising Difficulty of Natural Yes/No Questions
https://arxiv.org/abs/1905.10044

Courpus (train-dev split) is available in the repository of the BoolQ project: https://github.com/google-research-datasets/boolean-questions

In [1]:
!gsutil cp gs://boolq/train.jsonl .
!gsutil cp gs://boolq/dev.jsonl .

Copying gs://boolq/train.jsonl...
- [1 files][  6.2 MiB/  6.2 MiB]                                                
Operation completed over 1 objects/6.2 MiB.                                      
Copying gs://boolq/dev.jsonl...
- [1 files][  2.1 MiB/  2.1 MiB]                                                
Operation completed over 1 objects/2.1 MiB.                                      


## Part 1: EDA

In [2]:
import json
import pandas as pd

# Reading data
with open('train.jsonl') as f:
    train_json = list(f)

with open('dev.jsonl') as f:
    test_json = list(f)

train_lst = [json.loads(json_str) for json_str in train_json]
test_lst = [json.loads(json_str) for json_str in test_json]

train_df = pd.DataFrame(train_lst)
test_df = pd.DataFrame(test_lst)
full_df = pd.concat([train_df, test_df])

In [4]:
full_df.head()

,question,title,answer,passage
0,do iran and afghanistan speak the same language,Persian language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,Good Samaritan law,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,Windows Movie Maker,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,Powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,The Elder Scrolls Online,False,As with other games in The Elder Scrolls serie...


In [5]:
# Find yes/no ration in answers
full_df['answer'].value_counts()

True     7907
False    4790
Name: answer, dtype: int64

In [6]:
# Find average length of a question
avg_question_len = full_df['question'].apply(len).mean()
avg_question_len

43.78963534693234

In [7]:
# Find average length of a passage
avg_passage_len = full_df['passage'].apply(len).mean()
avg_passage_len

563.9233677246593

## Part 2: Baseline

In [9]:
from sklearn.metrics import accuracy_score

# Let's find test accuracy if we predict the most common answer on train (True)
test_answers = test_df['answer'].tolist()
pred_answers = [True] * len(test_answers)
accuracy_score(test_answers, pred_answers)

0.6217125382262997